In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext tensorboard

In [23]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Activation, Dropout, Dense, Input, Bidirectional, Layer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
import pickle
from tensorflow import keras
import keras.backend as kb

from tensorboard.plugins.hparams import api as hp


In [4]:
def creazione_modello_GloVe(filename):
    f = open(filename, encoding="utf8")
    embeding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeding_index[word] = coefs
    f.close()
    return embeding_index

In [5]:
dataset = pickle.load(open("drive/MyDrive/ml_project/dataset_IMDB.pickle", "rb"))

In [6]:
Y = np.array(list(map(lambda x: 1 if x=="positive" else 0, dataset["sentiment"])))

In [17]:
x_train,x_test,y_train,y_test = train_test_split(dataset["processed_review"],Y, test_size=0.1, shuffle=True)

In [35]:
files = open("drive/MyDrive/ml_project/IMDB/x_train.pickle", "wb")
pickle.dump(x_train, files)
files.close()

In [37]:
files = open("drive/MyDrive/ml_project/IMDB/x_test.pickle", "wb")
pickle.dump(x_test, files)
files.close()

In [39]:
files = open("drive/MyDrive/ml_project/IMDB/y_train.pickle", "wb")
pickle.dump(y_train, files)
files.close()

In [40]:
files = open("drive/MyDrive/ml_project/IMDB/y_test.pickle", "wb")
pickle.dump(y_test, files)
files.close()

In [7]:
x_train = pickle.load(open("drive/MyDrive/ml_project/IMDB/x_train.pickle", "rb"))
x_test = pickle.load(open("drive/MyDrive/ml_project/IMDB/x_test.pickle", "rb"))
y_train = pickle.load(open("drive/MyDrive/ml_project/IMDB/y_train.pickle", "rb"))
y_test = pickle.load(open("drive/MyDrive/ml_project/IMDB/y_test.pickle", "rb"))

In [8]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(45000,) (45000,)
(5000,) (5000,)


In [9]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  22463
numero di negativi nel train:  22537
numero di positivi nel test:  2537
numero di negativi nel test:  2463


In [10]:
embedding = creazione_modello_GloVe("drive/MyDrive/ml_project/glove.6B.50d.txt")

In [11]:
tokenizer = Tokenizer(num_words=142092)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [12]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=200, weights=[embedding_matrix])

In [14]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=kb.squeeze(kb.tanh(kb.dot(x,self.W)+self.b),axis=-1)
        at=kb.softmax(et)
        at=kb.expand_dims(at,axis=-1)
        output=x*at
        return kb.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [15]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=200)

In [16]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)
y_test_categorical = keras.utils.to_categorical(y_test, 2)

In [17]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=200)

In [18]:
def train_test_model(hparams):
    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(units=hparams[HP_NUM_UNITS], return_sequences=True, dropout=hparams[HP_DROPOUT])),
        Dense(hparams[HP_NUM_UNITS]),
        Dense(hparams[HP_NUM_UNITS]),
        attention(),
        Dense(2, activation="sigmoid"),
    ])
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=8, batch_size=128, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]), callbacks=[tensorflow.keras.callbacks.TensorBoard(log_dir), hp.KerasCallback(log_dir, hparams)])
    _, accuracy = model.evaluate(x_test_index, y_test_categorical)
    y_pred = np.argmax(model.predict(x_test_index), axis=1).astype("float32")
    print(classification_report(y_test, y_pred))
    model.save("drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/{}_{}_{}".format(hparams[HP_NUM_UNITS], hparams[HP_DROPOUT], hparams[HP_OPTIMIZER]))
    return accuracy, precision_score(y_test, y_pred), recall_score(y_test, y_pred)

In [19]:
def run(run_dir, hparams):
    with tensorflow.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy, precision, recall = train_test_model(hparams)
        tensorflow.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        tensorflow.summary.scalar(METRIC_PRECISION, precision, step=1)
        tensorflow.summary.scalar(METRIC_RECALL, recall, step=1)

In [20]:
n_esecuzione = 0

In [21]:
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(["adam", "rmsprop"]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64]))

log_dir = 'drive/MyDrive/ml_project/IMDB/logs/IMDB_BiGRUAtt_HP_complete'
METRIC_ACCURACY = 'accuracy'
METRIC_PRECISION = "precision"
METRIC_RECALL = "recall"

with tensorflow.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy"), hp.Metric(METRIC_PRECISION, display_name="Precision"), hp.Metric(METRIC_RECALL, display_name="Recall")],
    )

In [47]:
#3 ore 59 min
for num_units in HP_NUM_UNITS.domain.values:
        for dropout in HP_DROPOUT.domain.values:
            for opt in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_DROPOUT: dropout,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: opt,
                    }
                print("esecuzione: ", n_esecuzione)
                print({h.name: hparams[h] for h in hparams})
                run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
                n_esecuzione += 1 

esecuzione:  0
{'dropout': 0.1, 'num_units': 16, 'optimizer': 'adam'}
Epoch 1/10
274/274 [==============================] - 83s 288ms/step - loss: 0.4806 - accuracy: 0.7608 - val_loss: 0.3343 - val_accuracy: 0.8569
Epoch 2/10
274/274 [==============================] - 76s 277ms/step - loss: 0.2664 - accuracy: 0.8897 - val_loss: 0.2804 - val_accuracy: 0.8862
Epoch 3/10
274/274 [==============================] - 77s 282ms/step - loss: 0.1936 - accuracy: 0.9237 - val_loss: 0.2892 - val_accuracy: 0.8869
Epoch 4/10
274/274 [==============================] - 77s 283ms/step - loss: 0.1313 - accuracy: 0.9519 - val_loss: 0.3124 - val_accuracy: 0.8931
Epoch 5/10
274/274 [==============================] - 78s 285ms/step - loss: 0.0751 - accuracy: 0.9742 - val_loss: 0.3838 - val_accuracy: 0.8883
Epoch 6/10
274/274 [==============================] - 78s 284ms/step - loss: 0.0406 - accuracy: 0.9871 - val_loss: 0.4867 - val_accuracy: 0.8810
Epoch 7/10
274/274 [==============================] - 77s 28

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.1_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.1_adam/assets


esecuzione:  1
{'dropout': 0.1, 'num_units': 16, 'optimizer': 'rmsprop'}
Epoch 1/10
274/274 [==============================] - 77s 266ms/step - loss: 0.2331 - accuracy: 0.9187 - val_loss: 0.4661 - val_accuracy: 0.8712
Epoch 2/10
274/274 [==============================] - 71s 257ms/step - loss: 0.0437 - accuracy: 0.9844 - val_loss: 0.6291 - val_accuracy: 0.8746
Epoch 3/10
274/274 [==============================] - 71s 259ms/step - loss: 0.0216 - accuracy: 0.9927 - val_loss: 0.7453 - val_accuracy: 0.8741
Epoch 4/10
274/274 [==============================] - 76s 277ms/step - loss: 0.0138 - accuracy: 0.9950 - val_loss: 0.8192 - val_accuracy: 0.8760
Epoch 5/10
274/274 [==============================] - 76s 277ms/step - loss: 0.0101 - accuracy: 0.9962 - val_loss: 0.9145 - val_accuracy: 0.8726
Epoch 6/10
274/274 [==============================] - 77s 281ms/step - loss: 0.0071 - accuracy: 0.9976 - val_loss: 1.0534 - val_accuracy: 0.8748
Epoch 7/10
274/274 [==============================] - 68s

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.1_rmsprop/assets


...

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.9_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.9_adam/assets


esecuzione:  17
{'dropout': 0.9, 'num_units': 16, 'optimizer': 'rmsprop'}
Epoch 1/10
274/274 [==============================] - 79s 268ms/step - loss: 0.2985 - accuracy: 0.8876 - val_loss: 0.5326 - val_accuracy: 0.8837
Epoch 2/10
274/274 [==============================] - 73s 266ms/step - loss: 0.1060 - accuracy: 0.9591 - val_loss: 0.6267 - val_accuracy: 0.8830
Epoch 3/10
274/274 [==============================] - 72s 263ms/step - loss: 0.0797 - accuracy: 0.9691 - val_loss: 0.6739 - val_accuracy: 0.8835
Epoch 4/10
274/274 [==============================] - 71s 260ms/step - loss: 0.0719 - accuracy: 0.9714 - val_loss: 0.6459 - val_accuracy: 0.8827
Epoch 5/10
274/274 [==============================] - 70s 255ms/step - loss: 0.0643 - accuracy: 0.9760 - val_loss: 0.6466 - val_accuracy: 0.8857
Epoch 6/10
274/274 [==============================] - 71s 260ms/step - loss: 0.0617 - accuracy: 0.9769 - val_loss: 0.6492 - val_accuracy: 0.8861
Epoch 7/10
274/274 [==============================] - 70

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.9_rmsprop/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/16_0.9_rmsprop/assets


In [20]:
n_esecuzione = 18
for num_units in HP_NUM_UNITS.domain.values:
        for dropout in HP_DROPOUT.domain.values:
            for opt in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_DROPOUT: dropout,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: opt,
                    }
                print("esecuzione: ", n_esecuzione)
                print({h.name: hparams[h] for h in hparams})
                run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
                n_esecuzione += 1 

esecuzione:  18
{'dropout': 0.1, 'num_units': 32, 'optimizer': 'adam'}
Epoch 1/8
274/274 [==============================] - 34s 89ms/step - loss: 0.4329 - accuracy: 0.7911 - val_loss: 0.3099 - val_accuracy: 0.8680
Epoch 2/8
274/274 [==============================] - 22s 80ms/step - loss: 0.2467 - accuracy: 0.9006 - val_loss: 0.2550 - val_accuracy: 0.8950
Epoch 3/8
274/274 [==============================] - 22s 79ms/step - loss: 0.1704 - accuracy: 0.9352 - val_loss: 0.2665 - val_accuracy: 0.8948
Epoch 4/8
274/274 [==============================] - 22s 79ms/step - loss: 0.1033 - accuracy: 0.9623 - val_loss: 0.3564 - val_accuracy: 0.8818
Epoch 5/8
274/274 [==============================] - 21s 78ms/step - loss: 0.0542 - accuracy: 0.9810 - val_loss: 0.3975 - val_accuracy: 0.8884
Epoch 6/8
274/274 [==============================] - 21s 78ms/step - loss: 0.0292 - accuracy: 0.9907 - val_loss: 0.5015 - val_accuracy: 0.8826
Epoch 7/8
274/274 [==============================] - 22s 79ms/step - lo

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.1_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.1_adam/assets


esecuzione:  19
{'dropout': 0.1, 'num_units': 32, 'optimizer': 'rmsprop'}
Epoch 1/8
274/274 [==============================] - 17s 52ms/step - loss: 0.2105 - accuracy: 0.9128 - val_loss: 0.3610 - val_accuracy: 0.8892
Epoch 2/8
274/274 [==============================] - 11s 42ms/step - loss: 0.0517 - accuracy: 0.9810 - val_loss: 0.5138 - val_accuracy: 0.8845
Epoch 3/8
274/274 [==============================] - 12s 43ms/step - loss: 0.0283 - accuracy: 0.9896 - val_loss: 0.6044 - val_accuracy: 0.8814
Epoch 4/8
274/274 [==============================] - 12s 43ms/step - loss: 0.0187 - accuracy: 0.9934 - val_loss: 0.7149 - val_accuracy: 0.8790
Epoch 5/8
274/274 [==============================] - 11s 41ms/step - loss: 0.0117 - accuracy: 0.9958 - val_loss: 0.8791 - val_accuracy: 0.8618
Epoch 6/8
274/274 [==============================] - 11s 41ms/step - loss: 0.0104 - accuracy: 0.9963 - val_loss: 0.7784 - val_accuracy: 0.8799
Epoch 7/8
274/274 [==============================] - 11s 42ms/step -

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.1_rmsprop/assets


...

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.9_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.9_adam/assets


esecuzione:  35
{'dropout': 0.9, 'num_units': 32, 'optimizer': 'rmsprop'}
Epoch 1/8
274/274 [==============================] - 16s 48ms/step - loss: 0.1973 - accuracy: 0.9224 - val_loss: 0.7147 - val_accuracy: 0.8880
Epoch 2/8
274/274 [==============================] - 12s 43ms/step - loss: 0.0904 - accuracy: 0.9665 - val_loss: 0.6991 - val_accuracy: 0.8861
Epoch 3/8
274/274 [==============================] - 11s 40ms/step - loss: 0.0779 - accuracy: 0.9696 - val_loss: 0.7305 - val_accuracy: 0.8853
Epoch 4/8
274/274 [==============================] - 11s 41ms/step - loss: 0.0727 - accuracy: 0.9727 - val_loss: 0.6581 - val_accuracy: 0.8877
Epoch 5/8
274/274 [==============================] - 11s 41ms/step - loss: 0.0676 - accuracy: 0.9745 - val_loss: 0.6559 - val_accuracy: 0.8862
Epoch 6/8
274/274 [==============================] - 11s 41ms/step - loss: 0.0639 - accuracy: 0.9762 - val_loss: 0.7036 - val_accuracy: 0.8888
Epoch 7/8
274/274 [==============================] - 11s 40ms/step -

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.9_rmsprop/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/32_0.9_rmsprop/assets


In [23]:
n_esecuzione = 36
for num_units in HP_NUM_UNITS.domain.values:
        for dropout in HP_DROPOUT.domain.values:
            for opt in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_DROPOUT: dropout,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: opt,
                    }
                print("esecuzione: ", n_esecuzione)
                print({h.name: hparams[h] for h in hparams})
                run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
                n_esecuzione += 1 

esecuzione:  36
{'dropout': 0.1, 'num_units': 64, 'optimizer': 'adam'}
Epoch 1/8
274/274 [==============================] - 28s 89ms/step - loss: 0.0490 - accuracy: 0.9877 - val_loss: 0.8588 - val_accuracy: 0.8808
Epoch 2/8
274/274 [==============================] - 22s 81ms/step - loss: 0.0019 - accuracy: 0.9994 - val_loss: 0.9260 - val_accuracy: 0.8791
Epoch 3/8
274/274 [==============================] - 23s 83ms/step - loss: 8.5530e-04 - accuracy: 0.9998 - val_loss: 1.1302 - val_accuracy: 0.8744
Epoch 4/8
274/274 [==============================] - 22s 81ms/step - loss: 0.0013 - accuracy: 0.9995 - val_loss: 1.0771 - val_accuracy: 0.8794
Epoch 5/8
274/274 [==============================] - 22s 81ms/step - loss: 1.9602e-04 - accuracy: 1.0000 - val_loss: 1.2734 - val_accuracy: 0.8764
Epoch 6/8
274/274 [==============================] - 22s 80ms/step - loss: 5.1394e-05 - accuracy: 1.0000 - val_loss: 1.4190 - val_accuracy: 0.8750
Epoch 7/8
274/274 [==============================] - 22s 79

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.1_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.1_adam/assets


esecuzione:  37
{'dropout': 0.1, 'num_units': 64, 'optimizer': 'rmsprop'}
Epoch 1/8
274/274 [==============================] - 17s 49ms/step - loss: 0.0386 - accuracy: 0.9875 - val_loss: 0.9497 - val_accuracy: 0.8826
Epoch 2/8
274/274 [==============================] - 12s 44ms/step - loss: 0.0029 - accuracy: 0.9994 - val_loss: 1.5941 - val_accuracy: 0.8782
Epoch 3/8
274/274 [==============================] - 11s 41ms/step - loss: 5.6394e-04 - accuracy: 0.9998 - val_loss: 1.8310 - val_accuracy: 0.8795
Epoch 4/8
274/274 [==============================] - 11s 42ms/step - loss: 8.7781e-04 - accuracy: 0.9998 - val_loss: 2.0018 - val_accuracy: 0.8774
Epoch 5/8
274/274 [==============================] - 11s 41ms/step - loss: 2.9421e-04 - accuracy: 0.9999 - val_loss: 2.9894 - val_accuracy: 0.8782
Epoch 6/8
274/274 [==============================] - 11s 41ms/step - loss: 1.0698e-04 - accuracy: 0.9999 - val_loss: 3.6775 - val_accuracy: 0.8736
Epoch 7/8
274/274 [==============================] -

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.1_rmsprop/assets


...

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.9_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.9_adam/assets


esecuzione:  53
{'dropout': 0.9, 'num_units': 64, 'optimizer': 'rmsprop'}
Epoch 1/8
274/274 [==============================] - 17s 52ms/step - loss: 0.1062 - accuracy: 0.9593 - val_loss: 0.9932 - val_accuracy: 0.8844
Epoch 2/8
274/274 [==============================] - 12s 44ms/step - loss: 0.0433 - accuracy: 0.9834 - val_loss: 0.9930 - val_accuracy: 0.8829
Epoch 3/8
274/274 [==============================] - 12s 42ms/step - loss: 0.0381 - accuracy: 0.9857 - val_loss: 0.8897 - val_accuracy: 0.8787
Epoch 4/8
274/274 [==============================] - 11s 42ms/step - loss: 0.0361 - accuracy: 0.9860 - val_loss: 0.9649 - val_accuracy: 0.8777
Epoch 5/8
274/274 [==============================] - 12s 42ms/step - loss: 0.0367 - accuracy: 0.9866 - val_loss: 0.8911 - val_accuracy: 0.8836
Epoch 6/8
274/274 [==============================] - 11s 42ms/step - loss: 0.0325 - accuracy: 0.9878 - val_loss: 0.8667 - val_accuracy: 0.8832
Epoch 7/8
274/274 [==============================] - 12s 42ms/step -

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.9_rmsprop/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete/64_0.9_rmsprop/assets


RMSProp, 16, 0.9

In [24]:
HP_LR = hp.HParam('learning_rate', hp.Discrete([0.0001, 0.001, 0.005, 0.009, 0.1, 0.2, 0.25, 0.3]))


log_dir = 'drive/MyDrive/ml_project/IMDB/logs/IMDB_BiGRUAtt_HP_complete_LR'
METRIC_ACCURACY = 'accuracy'
METRIC_PRECISION = "precision"
METRIC_RECALL = "recall"

with tensorflow.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LR],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy"), hp.Metric(METRIC_PRECISION, display_name="Precision"), hp.Metric(METRIC_RECALL, display_name="Recall")],
    )

In [25]:
def run(run_dir, hparams):
    with tensorflow.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy, precision, recall = train_test_model(hparams)
        tensorflow.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        tensorflow.summary.scalar(METRIC_PRECISION, precision, step=1)
        tensorflow.summary.scalar(METRIC_RECALL, recall, step=1)

In [26]:
def train_test_model(hparams):
    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(units=16, return_sequences=True, dropout=0.9)),
        Dense(16),
        Dense(16),
        attention(),
        Dense(2, activation="sigmoid"),
    ])
    opt = RMSprop(learning_rate=hparams[HP_LR])
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=8, batch_size=128, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]), callbacks=[tensorflow.keras.callbacks.TensorBoard(log_dir), hp.KerasCallback(log_dir, hparams)])
    _, accuracy = model.evaluate(x_test_index, y_test_categorical)
    y_pred = np.argmax(model.predict(x_test_index), axis=1).astype("float32")
    print(classification_report(y_test, y_pred))
    model.save("drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/{}_{}_{}".format(16, 0.9, hparams[HP_LR]))
    return accuracy, precision_score(y_test, y_pred), recall_score(y_test, y_pred)

In [27]:
n_esecuzione = 0
for lr in HP_LR.domain.values:
    hparams = {
        HP_LR: lr,
        }
    print("esecuzione: ", n_esecuzione)
    print({h.name: hparams[h] for h in hparams})
    run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
    n_esecuzione += 1

esecuzione:  0
{'learning_rate': 0.0001}
Epoch 1/8
274/274 [==============================] - 17s 51ms/step - loss: 0.6863 - accuracy: 0.5882 - val_loss: 0.6644 - val_accuracy: 0.7582
Epoch 2/8
274/274 [==============================] - 11s 42ms/step - loss: 0.6696 - accuracy: 0.6595 - val_loss: 0.6188 - val_accuracy: 0.7480
Epoch 3/8
274/274 [==============================] - 12s 43ms/step - loss: 0.6378 - accuracy: 0.7052 - val_loss: 0.5579 - val_accuracy: 0.7362
Epoch 4/8
274/274 [==============================] - 11s 41ms/step - loss: 0.5922 - accuracy: 0.7248 - val_loss: 0.5250 - val_accuracy: 0.7414
Epoch 5/8
274/274 [==============================] - 11s 41ms/step - loss: 0.5463 - accuracy: 0.7422 - val_loss: 0.5229 - val_accuracy: 0.7645
Epoch 6/8
274/274 [==============================] - 12s 42ms/step - loss: 0.5128 - accuracy: 0.7566 - val_loss: 0.4987 - val_accuracy: 0.7943
Epoch 7/8
274/274 [==============================] - 11s 42ms/step - loss: 0.4977 - accuracy: 0.7643 

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.0001/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.0001/assets


esecuzione:  1
{'learning_rate': 0.001}
Epoch 1/8
274/274 [==============================] - 17s 50ms/step - loss: 0.5345 - accuracy: 0.7389 - val_loss: 0.4522 - val_accuracy: 0.8297
Epoch 2/8
274/274 [==============================] - 11s 42ms/step - loss: 0.4280 - accuracy: 0.8068 - val_loss: 0.3637 - val_accuracy: 0.8579
Epoch 3/8
274/274 [==============================] - 11s 41ms/step - loss: 0.3957 - accuracy: 0.8226 - val_loss: 0.3765 - val_accuracy: 0.8608
Epoch 4/8
274/274 [==============================] - 11s 41ms/step - loss: 0.3750 - accuracy: 0.8313 - val_loss: 0.3497 - val_accuracy: 0.8706
Epoch 5/8
274/274 [==============================] - 11s 42ms/step - loss: 0.3545 - accuracy: 0.8431 - val_loss: 0.3382 - val_accuracy: 0.8725
Epoch 6/8
274/274 [==============================] - 11s 41ms/step - loss: 0.3433 - accuracy: 0.8495 - val_loss: 0.3162 - val_accuracy: 0.8801
Epoch 7/8
274/274 [==============================] - 11s 41ms/step - loss: 0.3305 - accuracy: 0.8558 -

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.001/assets


...

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.25/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.25/assets


esecuzione:  7
{'learning_rate': 0.3}
Epoch 1/8
274/274 [==============================] - 16s 47ms/step - loss: 20.9703 - accuracy: 0.5036 - val_loss: 1.3741 - val_accuracy: 0.5067
Epoch 2/8
274/274 [==============================] - 11s 41ms/step - loss: 10.4207 - accuracy: 0.5017 - val_loss: 5.3152 - val_accuracy: 0.4850
Epoch 3/8
274/274 [==============================] - 11s 41ms/step - loss: 12.5881 - accuracy: 0.5049 - val_loss: 11.0555 - val_accuracy: 0.4986
Epoch 4/8
274/274 [==============================] - 11s 41ms/step - loss: 9.0562 - accuracy: 0.5031 - val_loss: 6.8200 - val_accuracy: 0.4834
Epoch 5/8
274/274 [==============================] - 11s 41ms/step - loss: 11.2121 - accuracy: 0.5012 - val_loss: 5.4638 - val_accuracy: 0.4679
Epoch 6/8
274/274 [==============================] - 11s 41ms/step - loss: 11.1479 - accuracy: 0.5055 - val_loss: 2.3848 - val_accuracy: 0.5188
Epoch 7/8
274/274 [==============================] - 11s 41ms/step - loss: 8.7530 - accuracy: 0.50

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.3/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/IMDB/models/IMDB_BiGRUAtt_HP_complete_LR/16_0.9_0.3/assets
